In [1]:
from __future__ import print_function
import cv2
import argparse
import time
import numpy as np

from CentroidTracker import CentroidTracker, progressBar, showWindows, georeference
import params
from collections import defaultdict
from math import sqrt
import os
import matplotlib.pyplot as plt
import time


from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.color import rgb2gray

In [2]:
input_video = 'input/video/amoebe_video.avi'
input_picture = cv2.imread('input/foto/cars.JPG')
output_video = 'output/video/amoebe_rand.avi'
path = defaultdict(list)

In [3]:
# Set basic variables
ct = CentroidTracker()
current_frame = 0
id = 1

In [4]:
backSub = cv2.createBackgroundSubtractorMOG2()

In [5]:
#Georeferencing
coord_pixel0_x = 0
coord_pixel0_y = 0
scale = 1

In [6]:
capture = cv2.VideoCapture(input_video)

# running the loop
# extracting the frames
ret, first_frame = capture.read()

# converting to gray-scale
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
marker_count, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

segmented = cv2.watershed(first_frame,markers)

# END of original watershed example

output = np.zeros_like(first_frame)


output = np.zeros_like(first_frame)
output2 = first_frame.copy()

# Iterate over all non-background labels
for i in range(2, marker_count + 1):
    mask = np.where(segmented==i, np.uint8(255), np.uint8(0))
    x,y,w,h = cv2.boundingRect(mask)
    area = cv2.countNonZero(mask[y:y+h,x:x+w])
    print("Label %d at (%d, %d) size (%d x %d) area %d pixels" % (i,x,y,w,h,area))

    # Visualize
    color = np.uint8(np.random.random_integers(0, 255, 3)).tolist()
    output[mask!=0] = color
    cv2.rectangle(output2, (x,y), (x+w,y+h), color, 1)
    cv2.putText(output2,'%d'%i,(x+w/4, y+h/2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

# cv2.imwrite('wshseg_colors.png', output)
# cv2.imwrite('wshseg_boxes.png', output2)


while True:
    current_frame += 1
    ret, frame = capture.read()

#     if frame is None:
#         print('finished!')
#         break
    
    
    
    
    
    
    
    
    
    cv2.imshow('Frame',output)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

Label 2 at (100, 1) size (923 x 281) area 222144 pixels


<ipython-input-6-c59b374eb344>:49: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  color = np.uint8(np.random.random_integers(0, 255, 3)).tolist()


TypeError: integer argument expected, got float

In [ ]:
#############----------- RUN ALL ABOVE ---------------